In [1]:

    #return np.exp(prediction) / (1. + np.exp(prediction))


In [7]:
def gradient_solver(N, eta, design, data, beta=None):
    M=len(data)
    if beta != None:
        beta = beta
    else:
        beta = np.random.randn(design.shape[1])
     
    for i in range(N):
        gradients = cost_grad_ols(design,frank,beta)
        beta -= eta*gradients
    return beta


In [8]:
import functions_class as fx
import classx as cl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score

In [200]:
import loaddata as ld

X,y = ld.load_data(scaler='minmax')



# Egen logistic regression.


In [221]:
eta = 0.1 # This is out eta

Niteration = 100
beta = np.random.randn(X.shape[1],1)
#
#beta = parameters.reshape([26,1])

for iter in range(Niteration):
    
    sig = sigmoid(X@beta)
    gradient = np.dot(X.T, (sig - y)) / y.shape[0]
    beta -= eta*gradient
    
    
    #Cost function
    cost = cost_log_ols(X@beta,y.T)
    # Log Loss function from sklearn
    logloss=log_loss(y, np.round(X@beta), eps=1e-16, normalize=True)
    print('Cost', cost,'&','Log loss', logloss)


Cost [[1.95375457]] & Log loss 23.777812541919044
Cost [[1.83905812]] & Log loss 23.591937335964303
Cost [[1.73223892]] & Log loss 23.176005173180876
Cost [[1.63365266]] & Log loss 22.505629414107197
Cost [[1.54350635]] & Log loss 22.221897452915492
Cost [[1.46182741]] & Log loss 21.42780627828449
Cost [[1.38845303]] & Log loss 20.61043452183692
Cost [[1.32304156]] & Log loss 19.903095413537404
Cost [[1.26510253]] & Log loss 18.562899265532373
Cost [[1.21403854]] & Log loss 17.7687000446192
Cost [[1.16919048]] & Log loss 17.37845575932302
Cost [[1.12987947]] & Log loss 16.345491732254533
Cost [[1.09544075]] & Log loss 14.888990265357053
Cost [[1.06524792]] & Log loss 14.300395139896143
Cost [[1.03872789]] & Log loss 13.950564820590545
Cost [[1.01536803]] & Log loss 13.604240883872752
Cost [[0.99471758]] & Log loss 13.27016951807273
Cost [[0.97638523]] & Log loss 13.083031413095654
Cost [[0.96003446]] & Log loss 12.824882768472968
Cost [[0.94537782]] & Log loss 12.646316583628819
Cost [

# Accuracy. 
Både egen kode og tester med scikit. 

In [205]:

activation =sigmoid(X@beta) 
classes = np.zeros([len(activation)])
classes=np.round(activation)
print(100*np.sum(classes==y)/len(activation),'%')


80.3 %


In [214]:
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import log_loss

model = LogisticRegression()
model.fit(X, y)
predicted_classes = model.predict(X)
accuracy = accuracy_score(y.flatten(),predicted_classes)
accuracy = accuracy * 100
parameters = model.coef_
log_loss(y, predicted_classes)

print(accuracy, '%')

C:\Users\vemundst\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\vemundst\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


82.15 %
